In [1]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.layers import MaxPooling2D,Flatten,Dense,Input,GlobalAveragePooling2D,Dropout
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense,AveragePooling2D
from keras.models import Sequential,Model
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3

from keras.layers import BatchNormalization
from keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
import cv2 as cv

In [2]:
data_dir='../input/face-dataset/Datasets/Train'
data_dir = pathlib.Path(data_dir)
data_dir

In [3]:
aaryan= list(data_dir.glob('Aaryan/*'))
arihant= list(data_dir.glob('Arihant/*'))
asmita= list(data_dir.glob('Asmita/*'))
jay= list(data_dir.glob('Jay/*'))
vijay= list(data_dir.glob('Vijay/*'))

In [4]:
len(aaryan),len(arihant),len(asmita),len(jay),len(vijay)


In [5]:
# Contains the images path
df_images = {
    'aaryan' : aaryan,
    'arihant' : arihant,
    'asmita' : asmita,
    'jay' : jay,
    'vijay':vijay
}

# Contains numerical labels for the categories
df_labels = {
    'aaryan' : 0,
    'arihant' : 1,
    'asmita' : 2,
    'jay':3,
    'vijay':4
}

In [6]:
X=[]
y=[]
for label, images in df_images.items():
    for image in images:
        img=cv.imread(str(image))
        img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
        X.append(img)
        y.append(df_labels[label])

In [7]:
len(X),len(y)

In [8]:
X=np.array(X)
y=np.array(y)

In [9]:
X.shape,y.shape

In [10]:
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.7,random_state=42)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape) 
print(y_valid.shape)
print(X_test.shape) 
print(y_test.shape)

In [11]:
plt.imshow(X_train[300])
plt.show()
print(y_train[300])

In [242]:
base_model=InceptionV3(include_top=False, weights='imagenet',input_shape=(224,224,3))

In [243]:
for i in base_model.layers:
    i.trainable=False   

In [244]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x = Dense(128, activation ='relu')(x)
x=Dropout(0.5)(x)
x = Dense(64, activation ='relu')(x)
pred=Dense(5,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=pred)

In [245]:
model.summary()

In [246]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics='accuracy')

In [250]:
datagen = ImageDataGenerator(rotation_range=20, brightness_range=[0.4,1.4], shear_range=0.2, zoom_range=0.2,horizontal_flip=True)
aug_iter = datagen.flow(X_train,y_train, batch_size=1)
fig, ax = plt.subplots(nrows=1, ncols=6, figsize=(15,15))
for i in range(6):
    image = next(aug_iter)[0].astype('uint8')
    image = np.squeeze(image)
    ax[i].imshow(image)
    ax[i].axis('off')

In [251]:
batch_size = 32
r=model.fit(aug_iter,epochs=15,validation_data=(X_valid,y_valid),verbose=1,steps_per_epoch=X_train.shape[0] // batch_size)

In [254]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [255]:
plt.plot(r.history['accuracy'], label='train accuracy')
plt.plot(r.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()

In [256]:
y_pred=model.predict(X_test).argmax(axis=1)
accuracy_score(y_test,y_pred)

In [257]:
from keras.models import load_model
model.save('vgg_model_88.h5')
from IPython.display import FileLink
FileLink(r'./vgg_model_88.h5')